In [ ]:
import os
import numpy as np
from pymooIN import *

In [ ]:
# checkpointDir = 'checkpoint' + '.npy'
checkpointDir = 'checkpoint.npy'

plotDir = './plots'

In [ ]:
checkpoint, = np.load(checkpointDir, allow_pickle=True).flatten()
algorithm = checkpoint

pf = algorithm.problem.pareto_front()
ps = algorithm.problem.pareto_set()

print('Number of individuals in final population: ' + str(len(algorithm.pop.get('X'))))
print('Number of generations: ', str(algorithm.n_gen), str(len(algorithm.callback.data['var'])), str(len(algorithm.callback.data['obj'])))

In [ ]:
print('FINAL POPULATION')
print('Parameters')
print(algorithm.pop.get('X'))
print('Objectives')
print(algorithm.pop.get('F'))

In [ ]:
print('EVERY GENERATION')
for gen in range(algorithm.n_gen):
    print(f'generation {gen}')
    var_g = algorithm.callback.data['var'][gen]
    obj_g = algorithm.callback.data['obj'][gen]
    for ind in range(len(var_g)):
        var_i = var_g[ind]
        obj_i = obj_g[ind]
        print(f'gen{gen} ind{ind}: ', end='')
        for n in range(len(var_i)):
            print(f'{var_labels[n]}: {var_i[n]}', end=' ')
        print(' // ', end='')
        for n in range(len(obj_i)):
            print(obj_labels[n] + ':' + '%.3f' % obj_i[n], end=' ')
        print()

#### Create Directory for Saving Plots

In [ ]:
try:
    os.mkdir(plotDir)
except OSError:
    print(plotDir + ' directory already exists')

### Scatter Plots

In [ ]:
from pymoo.visualization.scatter import Scatter

In [ ]:
# All design points
if algorithm.n_gen < 10:
    leg = True
else:
    leg = False
plot = Scatter(title='Entire Design Space', legend=leg, labels=var_labels)
for g in range(len(algorithm.callback.data['var'])):  # range(algorithm.n_gen)
    plot.add(algorithm.callback.data['var'][g][:], label='GEN %i' % g)
if ps is not None:
    plot.add(ps, plot_type="line", color="black", alpha=0.7)
plot.save(plotDir + '/entire_design_space.png')

# All objective points
plot = Scatter(title='Entire Objective Space', legend=leg, labels=obj_labels)
for g in range(len(algorithm.callback.data['var'])):
    plot.add(algorithm.callback.data['obj'][g][:], label='GEN %i' % g)
if pf is not None:
    plot.add(pf, plot_type="line", color="black", alpha=0.7)
plot.save(plotDir + '/entire_obj_space.png')

### Last 10 Generation of Parameter and Objective Spaces

In [ ]:
# Last 10 generations
# last 10 gens. design points
if algorithm.n_gen > 10:
    plot = Scatter(title='Last 10 Generations Design Space', legend=True, labels=var_labels)
    for g in range(algorithm.n_gen-10, algorithm.n_gen):  # algorithm.n_gen == len(algorithm.callback.data['var'])
        plot.add(algorithm.callback.data['var'][g][:], label='GEN %i' % g)
    if ps is not None:
        plot.add(ps, plot_type="line", color="black", alpha=0.7)
    plot.save(plotDir + '/final_10_design_space.png')
    # last 10 gens. objective points
    plot = Scatter(title='Last 10 Generations Objective Space', legend=True, labels=obj_labels)
    for g in range(len(algorithm.callback.data['obj'])-10, len(algorithm.callback.data['obj'])):  # range(algorithm.n_gen)
        plot.add(algorithm.callback.data['obj'][g][:], label='GEN %i' % g)
    if pf is not None:
        plot.add(pf, plot_type="line", color="black", alpha=0.7)
    plot.save(plotDir + '/final_10_obj_space.png')


#### Decision Making 

In [ ]:
if pf is not None:
    from pymoo.factory import get_decision_making

    dm = get_decision_making("high-tradeoff")

    I = dm.do(pf)

    plot = Scatter(title='Pareto Front: High Tradeoff Points')
    plot.add(pf, alpha=0.2)
    plot.add(pf[I], color="red", s=100)
    plot.save(plotDir + '/pf-high-tradeoff')

In [ ]:
# from pymoo.util.running_metric import RunningMetric

# running = RunningMetric(
#                         delta_gen=5,
#                         # n_plots=4,
#                         only_if_n_plots=True,
#                         key_press=False,
#                         do_show=True)

# running.notify(algorithm)